In [2]:
import os
import numpy as np

In [9]:
# Load cached data
data = np.load('../data/imagenet/imagenet-resnet152.npz')
example_paths = os.listdir('../data/imagenet/examples')
scores = data['arr_0']
labels = data['arr_1']

In [27]:
# Problem setup
n=1000 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage

In [38]:
# Split the softmax scores into calibration and validation sets (save the shuffling)
idx = np.array([1] * n + [0] * (scores.shape[0]-n)) > 0
np.random.shuffle(idx)
cal_scores, val_scores = scores[idx,:], scores[~idx,:]
cal_labels, val_labels = labels[idx], labels[~idx]

In [39]:
# Calculate conformal scores (the least Ambiguous Classifier score)
cal_conformal = 1-cal_scores[np.arange(n),cal_labels.astype(int)]
qhat = np.quantile(cal_conformal, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')

In [40]:
# Calculate prediction sets
val_scores >= 1-qhat

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])